In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [3]:
import sys
import numpy as np
import pandas as pd
import random
import collections

In [4]:
import re

In [5]:
import import_ipynb
import preprocessor

importing Jupyter notebook from preprocessor.ipynb


In [6]:
from nltk.tokenize import word_tokenize

In [7]:
from tqdm import tqdm

## Data

In [8]:
dir_path = '../../../Data/'
data = pd.read_excel(dir_path + '한국어_대화체_번역.xlsx' , engine='openpyxl')

In [9]:
data[['원문','번역문']].head()

,원문,번역문
0,이번 신제품 출시에 대한 시장의 반응은 어떤가요?,How is the market's reaction to the newly rele...
1,판매량이 지난번 제품보다 빠르게 늘고 있습니다.,The sales increase is faster than the previous...
2,그렇다면 공장에 연락해서 주문량을 더 늘려야겠네요.,"Then, we'll have to call the manufacturer and ..."
3,"네, 제가 연락해서 주문량을 2배로 늘리겠습니다.","Sure, I'll make a call and double the volume o..."
4,지난 회의 마지막에 논의했던 안건을 다시 볼까요?,Shall we take a look at the issues we discusse...


In [10]:
en_data = data['번역문']

In [11]:
vs = preprocessor.VocabSet(word_tokenize)
en_tokens = vs.tokens(en_data)

In [12]:
en_encoder = preprocessor.Encoder(en_data , word_tokenize , en_tokens)
en_encoded = en_encoder.encode()

## Context Data

In [24]:
con_size = 11

en_word2vec = []

In [25]:
for i , sen in enumerate(tqdm(en_encoded)) :
    
    if len(sen) < con_size :
        continue
        
    for j in range(len(sen) - con_size + 1) :
        en_con = sen[j:j+con_size]
        en_word2vec.append(en_con)
        
en_wordvec = random.shuffle(en_word2vec)

100%|██████████| 100000/100000 [00:01<00:00, 75183.86it/s]


In [27]:
en_word2vec = np.array(en_word2vec)

en_cen = en_word2vec[:,int(con_size/2)]
en_neigh = np.hstack([en_word2vec[:,:int(con_size/2)],en_word2vec[:,int(con_size/2)+1:]])

In [29]:
print('Data Shape \n')
print('Center Shape : {}'.format(en_cen.shape))
print('Neighbor Shape : {}'.format(en_neigh.shape))

Data Shape 

Center Shape : (704707,)
Neighbor Shape : (704707, 10)


## Word2Vec Model